In [4]:
import obstools as ot
import pprint as pp

names_to_aliases = ot.get_names_to_aliases("C://Users//zrfir//OneDrive//____OBSIDIAN//PUBLIC")
# 
    

In [5]:
ot.printd(names_to_aliases[0])


Mathematical Object.md: {'mathematical object', 'mathematical objects'}
Straight Line in the Plane.md: {'straight lines in the plane'}
Vector Equation of a Line.md: {'vector equation of a line'}
Vector Addition.md: {'vector addition'}
Scalar Multiplication.md: {'scalar multiplication'}
Dimension (Mathematics).md: {'dimensions'}
System of Linear Equations.md: set()
Matrix.md: {'matrices', 'matrix'}
Field - Applied Mathematics.md: {'applied mathematics', 'Applied mathematics'}
Field - Computing.md: {'computing'}
Field - Statistics.md: {'statistics', 'Statistics', 'STATISTICS', 'statistical analysis'}
Field - Probability Theory.md: {'probability theory', 'probability', 'Probability theory', 'probabilistic', '<---Home of Field'}
Field - Economics.md: {'economics'}
Field - Physics.md: {'physical theory', 'physics', 'physical', 'PHYSICS'}
Matrix Algebra.md: {'matrices', 'algebra of matrices', 'matrix algebra'}
Matrix Addition.md: {'add'}
Scalar Multiplication with a Matrix.md: {'multiply the

In [ ]:
import obstools as ot
from pathlib import Path
import json
import networkx as nx

class VaultExhaustive:
    def __init__(self, vault_dir = None, file_name = None, make_json = 0.5):
        if not vault_dir is None:
            self.vault_dir = Path(vault_dir)
            self.all_paths = ot.get_all_paths(vault_dir)
            self.names_to_paths = ot.get_names_to_paths(self.all_paths)
            names_to_links_results = ot.get_names_to_links(self.names_to_paths)
            self.names_to_links = names_to_links_results[0]
            self.reading_errors = names_to_links_results[1]
            self.json = None
            if not file_name is None:
                self.file_name = Path(file_name)
                if make_json == 0.5:
                    if self.file_name.is_file():
                        try: 
                            with open(self.file_name, 'r') as ofile:
                                self.json = json.load(ofile)
                            make_json = False
                        except:
                            make_json = True
                    else:
                        make_json = True
                if make_json:
                    self.dump_errors = ot.produce_json_of_vault(self.vault_dir, self.file_name)
                    with open(self.file_name, 'r') as ofile:
                        self.json = json.load(ofile)
            self.mapping = dict()
            if type(self.json) is dict:
                self.mapping = ot.get_mapping_from_json(self.json)

v = VaultExhaustive("C://Users//zrfir//OneDrive//__Obsidian//PUBLIC", "2022-02-02_Obsidian_2.json", 1)